In [1]:
from faker import Faker
import random
import hashlib
import bcrypt
from base64 import b64encode
import datetime
# fake = Faker('ru_RU')
fake = Faker()

In [2]:
# ! pip install faker

In [3]:
class Profile:
    i = 0
    def __init__(self,time=0):
        self.id = Profile.next_id()
        self.gender = random.choice(('M', 'F'))
        self.name = fake.name_male() if self.gender == 'M' else fake.name_female()
        self.birthday = fake.date_of_birth(minimum_age=12, maximum_age=80).isoformat()
        self.email = fake.email()
        self.phone = fake.msisdn()
        self.password = fake.password()
        
        self.hashed_password = bcrypt.hashpw(self.password.encode('UTF-8'), bcrypt.gensalt())
        # self.salt = fake.password(length=8)
        # self.hash = b64encode(hashlib.sha1((self.salt + self.password).encode('utf-8')).digest()).decode('utf-8')
        
        
        self.created = time  # fake.past_datetime().isoformat()
        self.intro = fake.paragraph(nb_sentences=3)
        avatarName = ''.join((s[0] for s in self.name.split(' ')))
        avatarStr = f'https://eu.ui-avatars.com/api/?name={avatarName}&background=random&size={{}}'
        self.avatarS = avatarStr.format(32)
        self.avatarL = avatarStr.format(128)
        self.postCount = 0
        self.photoCount = 0

    @classmethod
    def next_id(cls): cls.i += 1; return cls.i
        
    # def __str__(self):
    #     return (f'{self.id},{self.name},{self.gender},{self.birthday},{self.email},{self.phone},'
    #             f'{self.salt},{self.hash},{self.created},{self.intro},{self.avatarS},{self.avatarL},'
    #             f'{self.postCount},{self.photoCount}\n')
    
    def __str__(self):
        return (f'{self.id},{self.name},{self.gender},{self.birthday},{self.email},{self.phone},'
                f'{self.intro},{self.avatarS},{self.avatarL},{self.created}\n')
    
    def account_str(self):
        return (f'{self.id},{self.email},{self.hashed_password},,{self.created}\n')

class Post:
    i = 0
    def __init__(self,userId,time,atchType='none',atchId=0):
        self.id = Post.next_id()
        self.userId = userId
        self.created = time # date_time_between(start_date=SINCE_CREATED)
#         self.modified = fake.date_between_dates(date_start=self.created) if not random.randint(0,10) else None
        self.tags = '-'.join(fake.words(nb=random.randint(0,3)))
        self.content = fake.paragraph(nb_sentences=8)
        self.atchType = atchType
        self.atchId = atchId
        self.reaction = [0,0,0,0,0,0]
        self._cmt = []

    @classmethod
    def next_id(cls): cls.i += 1; return cls.i
    
    def __str__(self):
        if self.tags == '':
            self.tags = "''"
        return (f'{self.id},{self.userId},{self.created},{self.tags},{self.content},{self.atchType},'
                f'{self.atchId},"","{{{",".join(str(r) for r in self.reaction)}}}",{len(self._cmt)}\n')
#         .replace(',None,',',,').replace(',None\n',',\n')

class Comment:
    i = 0
    def __init__(self,userId,postId,created,parentId=0,replyCount=0):
        self.id = Comment.next_id()
        self.userId = userId
        self.postId = postId
        self.parentId = parentId
        self.content = fake.paragraph(nb_sentences=2)
        self.created = created

    @classmethod
    def next_id(cls): cls.i += 1; return cls.i
    
    def __str__(self):
        return (f'{self.id},{self.userId},{self.postId},{self.parentId},{self.content},{self.created}\n')

# class Reaction:
#     def __init__(self,userId,postId,t):
#         self.userId = userId
#         self.postId = postId
#         self.type = t
#     def __str__(self):
#         return (f'{self.userId},{self.postId},{self.type}\n')

class Relationship:
    def __init__(self,user1,user2,t,created):
        self.user1 = user1
        self.user2 = user2
        self.created = created
        self.type = t
        self.other = "''"
        if not random.randint(0,20):
            self.other = random.choices(('close friend','brother','sister','mother','father'), (4,2,2,1,1), k=1)[0]
    def __str__(self):
        return (f'{self.user1},{self.user2},{self.created},{self.type},{self.other}\n')

class Notification:
    i = 0
    def __init__(self,userId,t,created,fromUserId,postId=0,cmtId=0):
        self.id = Notification.next_id()
        self.userId = userId
        self.type = t
        self.created = created
        self.fromUserId = fromUserId
        self.postId = postId
        self.cmtId = cmtId

    @classmethod
    def next_id(cls): cls.i += 1; return cls.i
    
    def __str__(self):
        return (f'{self.id},{self.userId},{self.type},{self.created},{self.fromUserId},{self.postId},{self.cmtId}\n').replace(',None,',',,').replace(',None\n',',\n')

class Album:
    i = 0
    def __init__(self,userId,descr,created):
        self.id = Album.next_id()
        self.userId = userId
        self.descr = descr
        self.created = created

    @classmethod
    def next_id(cls): cls.i += 1; return cls.i
    
    def __str__(self):
        return (f'{self.id},{self.userId},{self.descr},{self.created}\n')

class Photo:
    i = 0
    def __init__(self,userId,created,albumId):
        self.id = Photo.next_id()
        self.userId = userId
        self.albumId = albumId
        self.url = f'https://loremflickr.com/{random.choice(("480/320","320/180"))}/all?random={random.randrange(100000)}'
        self.created = created

    @classmethod
    def next_id(cls): cls.i += 1; return cls.i
    
    def __str__(self):
        return (f'{self.id},{self.userId},{self.albumId},{self.url},{self.created}\n')

In [4]:
SINCE_CREATED = '-5y'

def varied_step_range(stop,stepiter):
    step = iter(stepiter)
    start = 1
    s = 1

    for i in range(stop):
        yield start
        try:
            s = next(step)
            start += s
        except:
            start += s


class User:
    def __init__(self, time):
        time = datetime.datetime.fromtimestamp(time).isoformat()
        self.profile = Profile(time)
        self.id = self.profile.id
        self.posts = []
        self.comments = []
        self.reactions = {}
        self.relationships = []
        self.notifications = []
        self.albums = [Album(self.id,'Avatar',time),Album(self.id,'Upload',time)]
        self.photos = [Photo(self.id, time, self.albums[1].id)]
        self.profile.photoCount += 1
        self.friends = []
        self.requests = []

    def make_friend(self, u, time):
        self.relationships.append(Relationship(self.id, u.id, 'friend', time))
        u.relationships.append(Relationship(u.id, self.id, 'friend', time))
        self.friends.append(u.id)
        u.friends.append(self.id)

    def is_friend(self, u):
        return self.id in u.friends

    def send_friend_request(self, user, time):
        if self.is_friend(user): return
        if self.id in user.requests: return
        if self.id == user.id:
            return
        time = datetime.datetime.fromtimestamp(time).isoformat()
        user.requests.append(self.id)
        user.relationships.append(Relationship(self.id, user.id, 'request', time))
        user.notifications.append(Notification(user.id, 'request', time, self.id))

    def accept_friend_request(self, user, time):
        if self.is_friend(user): return
        
        time = datetime.datetime.fromtimestamp(time).isoformat()

        if user.id in self.requests:
            for r in self.relationships:
                if user.id == r.user1 and r.type == 'request':
                    r.type == 'friend'
                    r.time = time
                    user.relationships.append(Relationship(user.id, self.id, 'friend', time))
                    user.notifications.append(Notification(user.id, 'accept', time, self.id))
                    self.friends.append(user.id)
                    user.friends.append(self.id)
                    return
        
        self.notifications.append(Notification(self.id, 'request', time, user.id))
        user.notifications.append(Notification(user.id, 'accept', time, self.id))
        self.make_friend(user, time)

    def post(self, time):
        time = datetime.datetime.fromtimestamp(time).isoformat()
        if not random.randint(0,3):
            self.photos.append(Photo(self.id, time, self.albums[1].id))
            self.profile.photoCount += 1
            self.posts.append(Post(self.id, time, 'photo', self.photos[-1].id))
        else:
            self.posts.append(Post(self.id, time))
        self.profile.postCount += 1

    def react(self, user):
        if user.profile.postCount == 0:
            return
#         postId = random.randint(0,user.profile.postCount-1)
#         post = user.posts[postId]
#         post = random.choices(user.posts, [*range(1,user.profile.postCount+1)])[0]
        post = random.choices(user.posts, [i for i in varied_step_range(user.profile.postCount,range(0,user.profile.postCount//4*3))])[0]
        key = f'{self.id},{post.id}'
        if not self.reactions.get(key):
            c = random.choices((0,1,2,3,4,5), (5,2,1,1,1,1), k=1)[0]
            self.reactions[key] = ('like','love','haha','wow','sad','angry')[c]
            post.reaction[c] += 1

    def comment(self, user, time):
        if user.profile.postCount == 0:
            return
        time = datetime.datetime.fromtimestamp(time).isoformat()
#         postId = random.randint(0,user.profile.postCount-1)
#         post = user.posts[postId]
#         post = random.choices(user.posts, [*range(1,user.profile.postCount+1)])[0]
        # if user.profile.postCount > 35:
        #     print(user.profile.postCount)
        #     print([i for i in varied_step_range(user.profile.postCount,range(0,user.profile.postCount//4*3))])
        post = random.choices(user.posts, [i for i in varied_step_range(user.profile.postCount,range(0,user.profile.postCount//4*3))])[0]
        if not random.randint(0,2) and post._cmt:
            parentId = random.choice(post._cmt)
            self.comments.append(Comment(self.id, post.id, time, parentId))
            post._cmt.append(self.comments[-1].id)
        else:
            self.comments.append(Comment(self.id, post.id, time))
            post._cmt.append(self.comments[-1].id)
            user.notifications.append(Notification(user.id, 'cmt', time, self.id, post.id, self.comments[-1].id))

In [5]:
class SocialNetwork:
    def generate(self, diff=100000, n=100):
        now = int(datetime.datetime.now().timestamp())
        t = now - diff
        users = []

        i = 1
        while i < n:
            t += random.randint(1,1000)
            users.append(User(t))
            i += 1

            if random.randint(1,6) == 1:
                random.choice(users).post(t)

        dt = 100
        while t < now:
            t += random.randint(1,dt)
            c = random.randint(1,1200)
            
            if c == 1:
                if dt > 10: dt -= 1
                users.append(User(t))
                i += 1
            elif c < 8:
                u = random.choice(users)
                random.choice(users).send_friend_request(u, t)
            elif c < 20:
                u = random.choice(users)
                random.choice(users).accept_friend_request(u, t)
            elif c < 30:
                random.choice(users).post(t)
            elif c < 500:
                u = random.choice(users)
                try:
                    users[random.choice(u.friends)-1].comment(u, t)
                except:
                    random.choice(users).comment(u, t)
            else:
                u = random.choice(users)
                try:
                    users[random.choice(u.friends)-1].react(u)
                except:
                    random.choice(users).react(u)
        
        self.users = users
        print(len(users))


    def save(self, save_dir='csv/'):
        f0 = open(save_dir+'pw_backup.csv', 'w')
        f1 = open(save_dir+'profile.csv', 'w')
        f2 = open(save_dir+'post.csv', 'w')
        f3 = open(save_dir+'comment.csv', 'w')
        f4 = open(save_dir+'reaction.csv', 'w')
        f5 = open(save_dir+'relationship.csv', 'w')
        f6 = open(save_dir+'notification.csv', 'w')
        f7 = open(save_dir+'album.csv', 'w')
        f8 = open(save_dir+'photo.csv', 'w')
        f9 = open(save_dir+'account.csv', 'w')

        f0.write('Id,Email,Password\n')
        # f1.write('Id,Name,Gender,Birthday,Email,Phone,Salt,Hash,Created,Intro,AvatarS,AvatarL,PostCount,PhotoCount\n')
        f1.write('Id,Name,Gender,Birthday,Email,Phone,Intro,AvatarS,AvatarL,Created\n')
        f2.write('Id,UserId,Created,Tags,Content,AtchType,AtchId,AtchUrl,Reaction,CmtCount\n')
        f3.write('Id,UserId,PostId,ParentId,Content,Created\n')
        f4.write('UserId,PostId,Type\n')
        f5.write('User1,User2,Created,Type,Other\n')
        f6.write('Id,UserId,Type,Created,FromUserId,PostId,CmtId\n')
        f7.write('Id,UserId,Description,Created\n')
        f8.write('Id,UserId,AlbumId,Url,Created\n')
        f9.write('Id,Email,HashedPassword,Role,Created\n')

        for user in self.users:
            f0.write(','.join((str(user.profile.id), user.profile.email, user.profile.password)) + '\n')
            f1.write(str(user.profile))
            f2.write(''.join(str(o) for o in user.posts))
            f3.write(''.join(str(o) for o in user.comments))
            for k, v in user.reactions.items():
                f4.write(f'{k},{v}\n')
            f5.write(''.join(str(o) for o in user.relationships))
            f6.write(''.join(str(o) for o in user.notifications))
            f7.write(''.join(str(o) for o in user.albums))
            f8.write(''.join(str(o) for o in user.photos))
            f9.write(user.profile.account_str())

        f0.close()
        f1.close()
        f2.close()
        f3.close()
        f4.close()
        f5.close()
        f6.close()
        f7.close()
        f8.close()
        f9.close()

In [6]:
import datetime
# %%time
Net = SocialNetwork()
Net.generate(500000)
Net.save()

104


In [7]:
# def varied_step_range(stop,stepiter):
#     step = iter(stepiter)
#     start = 1

#     for i in range(stop):
#         yield start
#         try:
#             start += next(step)
#         except:
#             start += 1

# [i for i in varied_step_range(10,[1,2,3,4])]